## 基于自注意力的序列到序列模型
除长期依赖问题外，基于循环神经网络的序列到序列模型的另一个不足是
无法并行计算。

为了提高并行计算效率以及捕捉长距离的依赖关系，
可以使用自注意模型来建立一个全连接的网络结构。

本文简单介绍一个典型的基于自注意力的序列到序列模型：
Transformer[Vaswaniet al., 2017]。

### 1.1 自注意力
对于一个向量序列$H = [h_1, · · · , h_T ] \in R^{d_h \times T}$，首先用自注意力模型来对
其进行编码。
$$  self-att(Q,K,V) = softmax(\frac {K^TQ}{\sqrt d_h} V)\\
Q = W_QH,K = W_kX,V = W_VX$$
其中 $d_h$ 是输入向量 $h_t$的维度，$W_Q \in R^{d_k \times d_h}
,W_k \in R^{d_k \times d_h} , W_v \in R^{d_v \times d_h}$

### 1.2 多头自注意力 
自注意力模型可以看作是在一个线性投影空间中建立向量H 之间交互关系，
为了提取更多得交互信息，采用多头分注意力，
在多个不同的投影空间中捕捉不同的交互信息。
$$MultiHead(H) = W_o [head1;...;head_M]\\
head_m = self-att(Q_m,K_m,V_m)\\
\forall m \in[1:M],Q_m =W^m_QH,K = W^m_kX,V = W^m_VX $$

其中 $W_o   \in R^{d_h \times Xd_v}$为输出投影矩阵，
$W^m_Q \in R^{d_k \times d_h}
,W_k^m \in R^{d_k \times d_h} , W^m_v \in R^{d_v \times d_h}$为投影矩阵
$m \in [1,M]$

### 基于自注意力模型的序列编码
对于一个序列 $x_{1:T}$,可以构建一个多层的多头自注意力来对其进行编码，
但由于自注意力模型忽略输入信息的位置信息，因此出事的输入序列中加入位置编码信息来修正。

对于一个输入序列 $x_{[1:T]}$
$$ H^{(0)} = [e_{x_1} \oplus p_1,...,e_{x_t} \oplus p_t]$$
其中$e_{x_t}$表示词 $x_t$ 的嵌入向量，$p_t$ 为位置t的向量表示

第 $l$层的隐状态 $H^{(l)}$为：
$$Z^{(l)}=norm (h^{(l-1)} + MultiHead(H^{(l-1)}))\\
H^{(l)} = norm(Z^{(l)}+FNN(Z^{(l)})) $$

其中 norm(x) 表示层归一化 ,
FNN(x) 表示逐位置的前馈神经网络。
对于输入序列中的每个位置上向量Z
$$FFN(z) = W_2ReLu(W_1Z+b_1) + b_2$$ 

基于自注意力模型的序列编码可以看作是一个全连接的前馈神经网络，第
l 层的每个位置都接受第l − 1层的所有位置的输出。不同的是，其连接权重是
通过注意力机制动态计算得到。
### Transformer
将自注意力模型应用在序列到序列任务中，其整个网络结构可以分为两
部分：

##### 编码器 
编码器只包含多层的自注意力模块，每一层都接受前一层的输出作为输
入。编码器的输入为序列 $x_{1:s}$,输出为一个向量序列 $H^e = [h_1^e,...,h_s^e]$
##### 解码器
解码器依是通过自回归的方式来生成目标序列。和编码器不同，解码器
可以由以下三个模块构成：
1. 自注意力模块：第t步时，先使用自注意力模型对已生成的前缀序列$y_{1:(t−1)}$
进行编码得到$H^d = [h_1^d,...,h_{(t-1)}^d]$。
在训练时，解码器的输入为整个目标序列，这时可以通过一个掩码（mask）来
阻止每个位置选择其后面的输入信息。
2. 解码器到编码器注意力模块：使用$h_{(t-1)}^d$作为查询向量，
通过注意力机制来从输入序列He 中选取有用的信息。
3. 逐位置的前馈神经网络：使用一个前馈神经网络来综合得到所有信息。
4. 将上述三个步骤重复多次，最后通过一个全连接前馈神经网络来计算输出
概率。
![在这里插入图片描述](https://img-blog.csdnimg.cn/20190529130130659.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3MxNDM0MDg4OTU4,size_16,color_FFFFFF,t_70)

参考：
https://nndl.github.io/
